In [1]:
pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import necessary libraries
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_dataset
import random
import numpy as np
from collections import defaultdict

# Set up device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load the dataset from Hugging Face
dataset = load_dataset("deccan-ai/insuranceQA-v2")

# Collect all unique answers from the train split for negative sampling
train_answers = list(set(dataset['train']['output']))

# Define a custom dataset class for training
class QADataset(Dataset):
    def __init__(self, dataset, answer_pool, num_negatives=5):
        """
        Initialize the QA dataset.

        Args:
            dataset: Hugging Face dataset split (e.g., train)
            answer_pool: List of all possible answers for negative sampling
            num_negatives: Number of negative samples per positive pair
        """
        self.dataset = dataset
        self.answer_pool = answer_pool
        self.num_negatives = num_negatives

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        question = self.dataset[idx]['input']
        positive_answer = self.dataset[idx]['output']
        # Sample negative answers, ensuring the positive isn't included
        negatives = random.sample(self.answer_pool, self.num_negatives)
        if positive_answer in negatives:
            negatives.remove(positive_answer)
            negatives.append(random.choice(self.answer_pool))
        # First answer is positive, followed by negatives
        answers = [positive_answer] + negatives
        return question, answers

Using device: cuda


README.md:   0%|          | 0.00/531 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/14.3M [00:00<?, ?B/s]

valid.jsonl:   0%|          | 0.00/2.29M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21325 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3354 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3308 [00:00<?, ? examples/s]

In [3]:
# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define the collate function to tokenize question-answer pairs
def collate_fn(batch):
    """
    Collate function to process a batch of data into tokenized inputs.

    Args:
        batch: List of (question, answers) tuples from the dataset

    Returns:
        Dictionary of tokenized inputs (input_ids, attention_mask, etc.)
    """
    all_inputs = []
    for question, answers in batch:
        for answer in answers:
            all_inputs.append((question, answer))
    inputs = tokenizer(
        [q for q, a in all_inputs],
        [a for q, a in all_inputs],
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors='pt'
    )
    return inputs

# Create training dataset and dataloader
train_dataset = QADataset(dataset['train'], train_answers, num_negatives=5)
train_loader = DataLoader(
    train_dataset,
    batch_size=2,  # Number of questions per batch (results in 12 inputs with 5 negatives)
    shuffle=True,
    collate_fn=collate_fn
)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
# Initialize the BERT model for sequence classification (single score output)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1).to(device)

# Define the optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Training loop
num_epochs = 3
num_negatives = 5  # Must match the dataset's num_negatives

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_idx, inputs in enumerate(train_loader):
        # Move inputs to device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        # Forward pass
        outputs = model(**inputs)
        scores = outputs.logits.squeeze(1)  # Shape: [batch_size * (num_negatives + 1)]
        # Calculate number of questions in the batch
        B = inputs['input_ids'].shape[0] // (num_negatives + 1)
        # Reshape scores to [B, num_negatives + 1]
        scores = scores.view(B, num_negatives + 1)
        # Compute log softmax over the answer scores for each question
        log_probs = torch.nn.functional.log_softmax(scores, dim=1)
        # Loss: Maximize the probability of the positive answer (index 0)
        loss = -log_probs[:, 0].mean()
        total_loss += loss.item()
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch {epoch+1}, Batch {batch_idx+1}, Loss: {loss.item():.4f}")
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} completed, Average Loss: {avg_loss:.4f}")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens 

Epoch 1, Batch 100, Loss: 0.0913


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch 1, Batch 200, Loss: 0.0079


...

Epoch 3, Batch 10500, Loss: 0.0000


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch 3, Batch 10600, Loss: 0.7450


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch 3 completed, Average Loss: 0.0585


In [5]:

# Define a directory in Kaggle to save the fine-tuned model and tokenizer
save_directory = "/kaggle/working/QA-Insurances"

# Save the fine-tuned model
model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")

Model and tokenizer saved to /kaggle/working/QA-Insurances
